In [5]:
# Packages

import os
import pandas as pd

In [6]:
# Trim pfam annotation

def trimAnnotPfam(path):
    curedFile = []
    with open(path) as f:
        contents = f.readlines()
    for i in contents:
        if 'STRG' in i:
            curedFile.append(i)
    geneNames=[] ; pfamAnnot= [] ; pfamCode = []
    separator = "PF"
    for i in range(len(curedFile)):
        gene = curedFile[i].split(" - ",1)[0]
        gene = gene.replace(".p1","")
        gene = gene.rsplit(".",1)[0]
        geneNames.append(gene)
        splitDash = curedFile[i].split(" - ",1)[1]
        splitAnnot = splitDash.split(separator,1)[0] ; splitAnnot = splitAnnot.strip()
        splitCode = splitDash.split(separator,1)[1] ; splitCode = splitCode.split(".",1)[0] ; splitCode = separator + splitCode
        pfamAnnot.append(splitAnnot) ; pfamCode.append(splitCode)
    dic={'genes':geneNames,'pfam_annotation':pfamAnnot,'pfam_code':pfamCode}
    mergeDf=pd.DataFrame(dic) 
    return mergeDf

pfam_annot = trimAnnotPfam('/Users/mmeynadier/Documents/Astroides/comparative_transcriptomics_astroides/data/net/8_functionalAnnotation/ast_all_domtbl.out') 

In [7]:
# Trim callicoblast cell expression of Stylophora

stGeneAnnot = pd.read_csv('/Users/mmeynadier/Documents/Astroides/comparative_transcriptomics_astroides/data/net/8_functionalAnnotation/Spis_gene_annotation.tsv',sep='\t')
stGeneAnnot = stGeneAnnot.rename(columns={"Spis10006_1":"genes"})

stCellTypeFC = pd.read_csv('/Users/mmeynadier/Documents/Astroides/comparative_transcriptomics_astroides/data/net/8_functionalAnnotation/Spis_adult_cell_type_gene_FC.tsv',sep='\t')
stCallicoblast = stCellTypeFC.loc[stCellTypeFC['calicoblast'] > 2]
stCallicoblast.index.name = 'genes'

stMerge = pd.merge(stGeneAnnot,stCallicoblast,on='genes',how='inner')
stMerge = stMerge.filter(['genes','CUB/'])
stMerge = stMerge.dropna()

annot = list(stMerge['CUB/'])
cureAnnot = []
for i in annot:
    a = i.split('/')[0]
    cureAnnot.append(a)
stMerge['CUB/'] = cureAnnot
stMerge = stMerge.rename(columns={"CUB/":"pfam_annotation"})
stMerge


,genes,pfam_annotation
1,Spis14986_1,Spond_N
7,Spis_XP_022778944_1,MORN
8,Spis_XP_022779261_1,bZIP_Maf
9,Spis_XP_022779543_1,Trypsin
10,Spis_XP_022779802_1,Lipase
...,...,...
149,Spis_XP_022807783_1,CH
150,Spis_XP_022807817_1,F5_F8_type_C
152,Spis_XP_022809492_1,Lectin_C
153,Spis_XP_022810032_1,I-set


In [8]:
pfam_annot_cured = []
annot = list(pfam_annot['pfam_annotation'])
for i in annot:
    a = i.split(' ')
    if len(a) > 1:
        pfam_annot_cured.append(a[1])
    else:
        pfam_annot_cured.append(" - ")
pfam_annot['pfam_annotation'] = pfam_annot_cured
pfam_annot

,genes,pfam_annotation,pfam_code
0,STRG.26188,1-cysPrx_C,PF10417
1,STRG.8355,1-cysPrx_C,PF10417
2,STRG.8355,1-cysPrx_C,PF10417
3,STRG.23285,1-cysPrx_C,PF10417
4,STRG.21056,14-3-3,PF00244
...,...,...,...
114173,STRG.38856,ZZ,PF00569
114174,STRG.38856,ZZ,PF00569
114175,STRG.34440,ZZ,PF00569
114176,STRG.16299,ZZ,PF00569


In [9]:
# Get rid of comments if you want to see annotation based on Stylophora callicoblasts expressed genes

#newAnnot = pd.merge(pfam_annot,stMerge,on='pfam_annotation',how='inner')
#pfam_annot =  newAnnot.drop_duplicates(subset='pfam_annotation',ignore_index=True)
#pfam_annot = pfam_annot.rename(columns={"genes_x":"genes"})
#pfam_annot

In [10]:
rPath = os.getcwd()

dataPath = rPath + "/../../data/net/7_deseq2/annotatedGenome/juvenile/DESeq2_results_juvenile_"

amb_VS_ext_path = dataPath+"amb_VS_ext.csv"
amb_VS_low_path = dataPath+"amb_VS_low.csv"
low_VS_ext_path = dataPath+"low_VS_ext.csv"
sp_VS_gm_path = dataPath+"sp_VS_gm.csv"
sp_amb_VS_gm_low_natSim_path = dataPath+"sp_amb_VS_gm_low_natSim.csv"

In [11]:
amb_VS_ext = pd.read_csv(amb_VS_ext_path,sep=',')
amb_VS_low = pd.read_csv(amb_VS_low_path,sep=',')
low_VS_ext = pd.read_csv(low_VS_ext_path,sep=',')
sp_VS_gm = pd.read_csv(sp_VS_gm_path,sep=',')
sp_amb_VS_gm_low_natSim = pd.read_csv(sp_amb_VS_gm_low_natSim_path,sep=',')

In [12]:
amb_VS_ext = amb_VS_ext.rename(columns={'Unnamed: 0':'genes'})
amb_VS_low = amb_VS_low.rename(columns={'Unnamed: 0':'genes'})
low_VS_ext = low_VS_ext.rename(columns={'Unnamed: 0':'genes'})
sp_VS_gm = sp_VS_gm.rename(columns={'Unnamed: 0':'genes'})
sp_amb_VS_gm_low_natSim = sp_amb_VS_gm_low_natSim.rename(columns={'Unnamed: 0':'genes'})

In [13]:
amb_VS_ext = amb_VS_ext[amb_VS_ext['padj']<0.05]
amb_VS_ext = amb_VS_ext[(amb_VS_ext['log2FoldChange']<-2) | (amb_VS_ext['log2FoldChange']>2)]
amb_VS_low = amb_VS_low[amb_VS_low['padj']<0.05]
amb_VS_low = amb_VS_low[(amb_VS_low['log2FoldChange']<-2) | (amb_VS_low['log2FoldChange']>2)]
low_VS_ext = low_VS_ext[low_VS_ext['padj']<0.05]
low_VS_ext = low_VS_ext[(low_VS_ext['log2FoldChange']<-2) | (low_VS_ext['log2FoldChange']>2)]
sp_VS_gm = sp_VS_gm[sp_VS_gm['padj']<0.05]
sp_VS_gm = sp_VS_gm[(sp_VS_gm['log2FoldChange']<-2) | (sp_VS_gm['log2FoldChange']>2)]
sp_amb_VS_gm_low_natSim = sp_amb_VS_gm_low_natSim[sp_amb_VS_gm_low_natSim['padj']<0.05]
sp_amb_VS_gm_low_natSim = sp_amb_VS_gm_low_natSim[(sp_amb_VS_gm_low_natSim['log2FoldChange']<-2) | (sp_amb_VS_gm_low_natSim['log2FoldChange']>2)]

In [14]:
amb_VS_low_annot = pd.merge(amb_VS_low,pfam_annot,how="inner",on="genes")
amb_VS_low_annot=  amb_VS_low_annot.drop_duplicates(subset='genes',ignore_index=False)
amb_VS_ext_annot = pd.merge(amb_VS_ext,pfam_annot,how="inner",on="genes")
amb_VS_ext_annot=  amb_VS_ext_annot.drop_duplicates(subset='genes',ignore_index=False)
low_VS_ext_annot = pd.merge(low_VS_ext,pfam_annot,how="inner",on="genes")
low_VS_ext_annot=  low_VS_ext_annot.drop_duplicates(subset='genes',ignore_index=False)
sp_VS_gm_annot = pd.merge(sp_VS_gm,pfam_annot,how="inner",on="genes")
sp_VS_gm_annot=  sp_VS_gm_annot.drop_duplicates(subset='genes',ignore_index=False)
sp_amb_VS_gm_low_natSim_annot = pd.merge(sp_amb_VS_gm_low_natSim,pfam_annot,how="inner",on="genes")
sp_amb_VS_gm_low_natSim_annot=  sp_amb_VS_gm_low_natSim_annot.drop_duplicates(subset='genes',ignore_index=False)

In [15]:
amb_VS_low_annot

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code
0,STRG.5926,55.533063,24.817650,2.359162,10.519689,7.010443e-26,1.016514e-21,MADF_DNA_bdg,PF10545
1,STRG.28429,25.302775,-4.792624,1.097773,-4.365767,1.266772e-05,3.673638e-02,RVT_1,PF00078


In [16]:
amb_VS_ext_annot

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code
0,STRG.24691,0.792276,16.433598,3.075270,5.343790,9.102300e-08,0.000440,Glycos_transf_1,PF00534
2,STRG.11235,0.562919,16.858515,3.546402,4.753696,1.997313e-06,0.007240,MAM,PF00629
6,STRG.41547,24.116283,7.109908,1.610944,4.413504,1.017107e-05,0.029496,IBR,PF01485
10,STRG.31729,20.401095,-2.564130,0.610105,-4.202769,2.636700e-05,0.047790,DNA_pol_alpha_N,PF12254


In [17]:
sp_VS_gm_annot

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code
0,STRG.38115,31.668438,-3.710366,0.742186,-4.999237,5.755772e-07,0.001012,p450,PF00067
1,STRG.35495,8.276218,-4.966905,1.037233,-4.788608,1.679418e-06,0.001771,SAM_1,PF00536
5,STRG.10297,32.205966,-3.743216,0.793856,-4.715235,2.414324e-06,0.002122,Myb_DNA-bind_5,PF13873
9,STRG.25710,31.871013,3.585821,0.811646,4.417962,9.963573e-06,0.007507,p450,PF00067
14,STRG.19078,8.746854,3.463888,0.803254,4.312319,1.615510e-05,0.010650,SAM_2,PF07647
15,STRG.18053,9.037805,2.569285,0.608240,4.224129,2.398671e-05,0.011464,SSF,PF00474
16,STRG.24410,6.881306,4.269900,1.015381,4.205217,2.608314e-05,0.011464,LicD,PF04991
18,STRG.6673,9.049021,3.017244,0.714706,4.221659,2.425106e-05,0.011464,BACK,PF07707
29,STRG.9214,11.653841,-3.670026,0.863587,-4.249748,2.140115e-05,0.011464,Exo5,PF09810
32,STRG.3315,22.663995,3.738124,0.942586,3.965817,7.314494e-05,0.027555,Ank_2,PF12796


In [18]:
sp_amb_VS_gm_low_natSim_annot

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code
0,STRG.15023,1229.466025,-9.332512,0.769466,-12.128560,7.454845e-34,4.099419e-30,MoaC,PF01967
4,STRG.39502,343.689600,-6.398709,0.664042,-9.636006,5.633722e-22,1.548992e-18,MMR_HSR1,PF01926
8,STRG.15032,117.775123,-7.310390,0.907453,-8.055947,7.886607e-16,1.445615e-12,DUF1647,PF07801
11,STRG.433,28.520009,4.447997,0.612977,7.256390,3.975574e-13,4.372337e-10,PID,PF00640
18,STRG.29636,53.583173,-4.123885,0.572102,-7.208307,5.665170e-13,5.192128e-10,Glutaredoxin,PF00462
22,STRG.32397,51.046233,-3.745379,0.534463,-7.007744,2.421909e-12,1.902583e-09,TPR_10,PF13374
50,STRG.15029,90.762185,-8.803885,1.317460,-6.682468,2.349516e-11,1.614999e-08,DUF1647,PF07801
53,STRG.5926,148.655464,-23.722728,3.678359,-6.449270,1.123903e-10,6.867049e-08,MADF_DNA_bdg,PF10545
54,STRG.28429,22.988915,8.245828,1.296300,6.361049,2.003803e-10,1.101891e-07,RVT_1,PF00078
55,STRG.26281,47.417040,9.291651,1.598704,5.811988,6.173528e-09,2.611402e-06,Rho_N,PF07498


In [19]:
# Get common genes 

In [20]:
common_amb = pd.merge(amb_VS_low_annot,amb_VS_ext_annot,how='inner',on='genes')
common_amb

,genes,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,pfam_annotation_x,pfam_code_x,baseMean_y,log2FoldChange_y,lfcSE_y,stat_y,pvalue_y,padj_y,pfam_annotation_y,pfam_code_y


In [21]:
common_low = pd.merge(amb_VS_low_annot,low_VS_ext_annot,how='inner',on='genes')
common_low

,genes,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,pfam_annotation_x,pfam_code_x,baseMean_y,log2FoldChange_y,lfcSE_y,stat_y,pvalue_y,padj_y,pfam_annotation_y,pfam_code_y
0,STRG.5926,55.533063,24.81765,2.359162,10.519689,7.010443e-26,1.016514e-21,MADF_DNA_bdg,PF10545,55.533063,-23.191146,3.237443,-7.163414,7.869249e-13,1.141041e-08,MADF_DNA_bdg,PF10545


In [22]:
common_ext = pd.merge(amb_VS_ext_annot,low_VS_ext_annot,how='inner',on='genes')
common_ext

,genes,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,pfam_annotation_x,pfam_code_x,baseMean_y,log2FoldChange_y,lfcSE_y,stat_y,pvalue_y,padj_y,pfam_annotation_y,pfam_code_y
0,STRG.24691,0.792276,16.433598,3.075270,5.343790,9.102300e-08,0.000440,Glycos_transf_1,PF00534,0.792276,17.632839,3.154722,5.589348,2.279244e-08,0.000110,Glycos_transf_1,PF00534
1,STRG.11235,0.562919,16.858515,3.546402,4.753696,1.997313e-06,0.007240,MAM,PF00629,0.562919,16.510034,3.711081,4.448848,8.633220e-06,0.020864,MAM,PF00629
2,STRG.41547,24.116283,7.109908,1.610944,4.413504,1.017107e-05,0.029496,IBR,PF01485,24.116283,6.989076,1.653223,4.227547,2.362530e-05,0.042821,IBR,PF01485


In [23]:
common_sp_VS_gm = pd.merge(sp_amb_VS_gm_low_natSim_annot,sp_VS_gm_annot,how='inner',on='genes')
common_sp_VS_gm

,genes,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,pfam_annotation_x,pfam_code_x,baseMean_y,log2FoldChange_y,lfcSE_y,stat_y,pvalue_y,padj_y,pfam_annotation_y,pfam_code_y
0,STRG.3315,22.151678,-4.848847,0.900851,-5.382517,7.345150e-08,0.000024,Ank_2,PF12796,22.663995,3.738124,0.942586,3.965817,0.000073,0.027555,Ank_2,PF12796
1,STRG.10297,30.941968,3.292232,0.918803,3.583176,3.394423e-04,0.029166,Myb_DNA-bind_5,PF13873,32.205966,-3.743216,0.793856,-4.715235,0.000002,0.002122,Myb_DNA-bind_5,PF13873
2,STRG.26205,20.047468,2.444594,0.727283,3.361270,7.758495e-04,0.049357,RPE65,PF03055,31.933408,-2.651279,0.725245,-3.655704,0.000256,0.049451,RPE65,PF03055
3,STRG.35495,10.830979,6.419071,1.910451,3.359978,7.794874e-04,0.049357,SAM_1,PF00536,8.276218,-4.966905,1.037233,-4.788608,0.000002,0.001771,SAM_1,PF00536
